In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv("../data_raw/lrf_public_file_review.tab", sep="\t", engine="python")

In [5]:
relevant_keys = {
    "L3_A": "In your own words, what is the greatest source of RISK TO YOUR SAFETY in your daily life?",
    "L3_B": "Other than what you just mentioned in your own words what is another MAJOR source of RISK TO YOUR SAFETY in your daily life?",
    "L5": "Do you think that climate change is a very serious threat, a somewhat serious threat, or not a threat at all to the people in this country in the next 20 years?"
}

In [6]:
survey_columns = df.columns[df.columns.str.startswith("L")].to_list()
drop_columns = [item for item in survey_columns if item not in list(relevant_keys.keys())]
df_relevant = df.drop(drop_columns, axis=1)

In [7]:
bins = [15, 18, 31, 65, 100]
labels = ["15 to 17", "18 to 30", "31 to 64", "65+"]

df_relevant["AgeGroup"] = pd.cut(df_relevant["Age"], bins=bins, labels=labels, right=False)

In [375]:
l3_a = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L3_A", values="projection_weight", aggfunc='sum').astype(int)
l3_a = l3_a.rename_axis(None, axis=1)
l3_a.columns = l3_a.columns.astype(str)

l3_b = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L3_B", values="projection_weight", aggfunc='sum').astype(int)
l3_b = l3_b.rename({" ": "99"}, axis=1)
l3_b = l3_b.rename_axis(None, axis=1)

l3 = l3_a + l3_a

l5 = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L5", values="projection_weight", aggfunc='sum').astype(int)
l5 = l5.drop([98, 99], axis=1)

In [385]:
import plotly.graph_objects as go

countries = ["United Kingdom", "Ireland", "France", "Germany", "Spain", "Netherlands"]

fig = go.Figure()
buttons = []
i = 0

for country in countries:
    current = l5.loc[country]
    current = current.rename({1: "Very serious threat", 2: "Somewhat serious threat", 3: "Not a threat at all"}, axis=1)

    for col in current:
        fig.add_trace(
            go.Bar(
                x=current.index.to_list(),
                y=current[col].to_list(),
                name=col,
                visible=(i == 0)
            )
        )

    args = [False] * len(countries) * len(current.columns)
    args[i*len(current.columns):(i*len(current.columns))+len(current.columns)] = [True] * len(current.columns)
    
    button = dict(label = country,
                  method = "update",
                  args=[{"visible": args}])
    
    buttons.append(button)
    
    i += 1

fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            x=1,
            y=1,
            buttons=buttons
        )
    ],
    barmode="stack"
)

# Set title
fig.update_layout(title_text=relevant_keys.get("L5"))

fig.show()

In [322]:
l5_total = l5.copy()
l5_total = l5_total.groupby("Country").sum()
l5_total["total"] = l5_total[1]/l5_total.sum(axis=1)

In [384]:
import pycountry

input_countries = l5_total.index.to_list()
countries = {country.name : country.alpha_2 for country in pycountry.countries}
codes = [countries.get(country) for country in input_countries]

In [8]:
df_relevant

,WPID_RANDOM,Country,Year,GlobalRegion,CountryIncomeLevel,WGT,projection_weight,Age,Gender,Education,IncomeFeelings,INCOME_5,Urbanicity,HouseholdSize,ChildrenInHousehold,L3_A,L3_B,L5,AgeGroup
0,111111576,Palestine,2019,11,2,0.841987,2079.386798,58,2,2,3,4,2,3,0,18,1,3,31 to 64
1,111111666,Botswana,2019,4,3,0.938971,1274.932764,25,2,2,5,2,2,2,2,1,98,98,18 to 30
2,111111887,Tajikistan,2019,7,1,0.619578,3298.585616,36,1,3,4,5,2,3,3,5,9,1,31 to 64
3,111114902,United Kingdom,2019,13,4,0.420952,22882.524653,45,2,3,1,3,1,2,1,1,18,2,31 to 64
4,111115212,Taiwan,2019,8,4,0.601208,12349.242676,29,2,3,2,5,2,1,0,7,3,1,18 to 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154190,211108235,Brazil,2019,5,3,0.480291,72326.452037,43,1,2,2,5,2,1,0,5,3,1,31 to 64
154191,211108380,Serbia,2019,14,3,0.600998,3264.106006,85,1,2,2,2,1,1,0,3,9,2,65+
154192,211108951,Slovakia,2019,12,4,1.199834,5114.058611,30,2,3,4,1,2,2,2,4,2,2,18 to 30
154193,211109671,India,2019,10,2,1.430093,409396.443098,18,1,2,3,5,1,2,0,19,,2,18 to 30
